In [5]:
import xarray as xr
import rioxarray
import numpy as np
import netCDF4
import os

input_file_path = r"G:\MJI\SKZ\Script\MSP_Data\Data\dataset-armor-3d-nrt-monthly_1702244021696.nc"
ouput_folder_path = r"G:\MJI\SKZ\Script\MSP_Data\Output\Seasonal\SeaTemp_Copernicus"

monthly_ds = xr.open_dataset(input_file_path, engine = 'netcdf4', decode_times= True, mask_and_scale = False)

masked_monthly_ds = monthly_ds.where(monthly_ds != monthly_ds['to'].attrs['_FillValue'])
actual_monlthy_ds = masked_monthly_ds*monthly_ds['to'].attrs['scale_factor'] + monthly_ds['to'].attrs['add_offset']

winter_months = [time.to_numpy() for time in monthly_ds.time if time.dt.month in [1, 2, 12]]
summer_months = [time.to_numpy() for time in monthly_ds.time if time.dt.month in [6, 7, 8]]

season_dict = {'Winter': winter_months, 'Summer': summer_months}

season_ds_list = []

for season in season_dict.keys():
    
    season_ds = actual_monlthy_ds.sel(time= season_dict[season], latitude = slice(16, 24), longitude = slice(81, 95))
    season_mean_ds = season_ds.mean(dim= 'time', skipna=True, keep_attrs=True)

    season_array = np.array([season], dtype = str)
    
    for var in season_mean_ds.data_vars:
        season_mean_ds[var] = season_mean_ds[var].expand_dims(season=season_array, axis=0)
        
    ds_name = '{}_ds'.format(season)
    ds_name = season_mean_ds
    season_ds_list.append(ds_name)

combined_sesason_ds = xr.concat(season_ds_list , dim = 'season')
season_name_array = combined_sesason_ds['season'].to_numpy()

for i, season_name in zip(range(0, len(combined_sesason_ds['season'])), season_name_array):
    
    season_temp_ds = combined_sesason_ds['to'][i,0,:,:]
    
    season_temp_ds.rio.write_crs("epsg:4326", inplace=True)
    season_temp_ds.rio.write_nodata(-9999, encoded=True, inplace=True)
    
    season_temp_ds.rio.to_raster(os.path.join(ouput_folder_path, 'Sea_Surface_Temp_{}_2022.tif'.format(season_name)))
    
combined_sesason_ds.to_netcdf(os.path.join(ouput_folder_path, 'Copernicus_Seasonal_Sea_Surface_Temperature.nc'))